In [2]:
pip install pywin32

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python312\lib\site-packages\midi-0.2.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import win32print

# Detect the number of available printers using win32print


# Get the list of available printers
printers = win32print.EnumPrinters(win32print.PRINTER_ENUM_LOCAL | win32print.PRINTER_ENUM_CONNECTIONS)

# Print the number of available printers
print(f"Number of available printers: {len(printers)}")

# Optionally, print the names of the printers
for i, printer in enumerate(printers, 1):
    print(f"{i}: {printer[2]}")

Number of available printers: 10
1: KONICA MINOLTA 205i
2: OneNote (Desktop) - Protected
3: OneNote (Desktop)
4: NPID5E410 (HP LaserJet Tank MFP 2606sdw)
5: Microsoft Print to PDF
6: HP44F8D2 (HP Smart Tank 660-670 series)
7: HP Smart Tank 660-670 series
8: HP LaserJet 1018
9: ECE_Dept_Printer_1 (HP LaserJet MFP M233dw)
10: Brother DCP-T820DW


### Minimum System Requirements

- **Operating System:** Windows (the `pywin32` and `win32print` modules are Windows-specific)
- **Python Version:** Python 3.x
- **Dependencies:** `pywin32` package (install via `pip install pywin32`)

### Bundling with Electron

- You can bundle Python scripts with Electron using tools like [Electron + Python Bridge](https://github.com/fyears/electron-python-example) or [eel](https://github.com/python-eel/Eel).
- The bundled executable **must run on Windows** and include a Python runtime with `pywin32` installed.
- **Limitations:**  
    - The code will **not run on macOS or Linux** because `win32print` is Windows-only.
    - All target systems must have the required printer drivers installed.

### Summary

You can bundle this code with an Electron app for Windows, but it will **not work on non-Windows systems**. Ensure all dependencies are included in your package.

In [9]:
!pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


    numpy (>=1.19.*) ; python_version >= "3.7"
           ~~~~~~~^

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\prakh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [ ]:
import time

# Mapping of JOB_STATUS_* bitmask values to human-readable strings
JOB_STATUS_MAP = {
    0x00000001: "Paused",
    0x00000002: "Error",
    0x00000004: "Deleting",
    0x00000008: "Spooling",
    0x00000010: "Printing",
    0x00000020: "Offline",
    0x00000040: "Paper Out",
    0x00000080: "Printed",
    0x00000100: "Deleted",
    0x00000200: "Blocked Device Queue",
    0x00000400: "User Intervention Required",
    0x00000800: "Restarted",
    0x00001000: "Complete",
    0x00002000: "Retained",
    0x00004000: "Rendering Locally"
}
target_printer_name = "HP Smart Tank 660-670 series"  # Replace with your printer name
def decode_job_status(status_code):
    if status_code is None:
        return "Unknown"
    statuses = [name for bit, name in JOB_STATUS_MAP.items() if status_code & bit]
    return ", ".join(statuses) if statuses else "None"

def get_jobs(printer_name):
    try:
        handle = win32print.OpenPrinter(printer_name)
        jobs = win32print.EnumJobs(handle, 0, 999, 1)
        win32print.ClosePrinter(handle)
        return jobs
    except Exception:
        return []

while True:
    jobs = get_jobs(target_printer_name)
    if jobs:
        job_id = jobs[0]['JobId']
        print(f"Job detected: ID {job_id}, tracking status...")
        while True:
            jobs = get_jobs(target_printer_name)
            job_ids = [job['JobId'] for job in jobs]
            if job_id not in job_ids:
                print(f"Job ID {job_id} completed or removed from queue.")
                break
            else:
                job_status = jobs[0]['Status']
                status_meaning = decode_job_status(job_status)
                print(f"Job ID {job_id} still in queue. Status: {job_status} ({status_meaning})")
            time.sleep(2)
    time.sleep(2)

KeyboardInterrupt: 

In [4]:
import time

target_printer_name = "HP Smart Tank 660-670 series"  # Replace with your printer name
JOB_STATUS_MAP = {
    0x00000001: "Paused",
    0x00000002: "Error",
    0x00000004: "Deleting",
    0x00000008: "Spooling",
    0x00000010: "Printing",
    0x00000020: "Offline",
    0x00000040: "Paper Out",
    0x00000080: "Printed",
    0x00000100: "Deleted",
    0x00000200: "Blocked Device Queue",
    0x00000400: "User Intervention Required",
    0x00000800: "Restarted",
    0x00001000: "Complete",
    0x00002000: "Retained",
    0x00004000: "Rendering Locally"
}
def decode_job_status(status_code):
    if status_code is None:
        return "Unknown"
    statuses = [name for bit, name in JOB_STATUS_MAP.items() if status_code is not None and status_code & bit]
    return ", ".join(statuses) if statuses else "None"

def decode_printer_status(status_code):
    # See: https://learn.microsoft.com/en-us/windows/win32/printdocs/printer-status
    PRINTER_STATUS_MAP = {
        0x00000000: "Ready",
        0x00000001: "Paused",
        0x00000002: "Error",
        0x00000004: "Pending Deletion",
        0x00000008: "Paper Jam",
        0x00000010: "Paper Out",
        0x00000020: "Manual Feed",
        0x00000040: "Paper Problem",
        0x00000080: "Offline",
        0x00000100: "IO Active",
        0x00000200: "Busy",
        0x00000400: "Printing",
        0x00000800: "Output Bin Full",
        0x00001000: "Not Available",
        0x00002000: "Waiting",
        0x00004000: "Processing",
        0x00008000: "Initializing",
        0x00010000: "Warming Up",
        0x00020000: "Toner Low",
        0x00040000: "No Toner",
        0x00080000: "Page Punt",
        0x00100000: "User Intervention",
        0x00200000: "Out of Memory",
        0x00400000: "Door Open",
        0x00800000: "Server Unknown",
        0x01000000: "Power Save"
    }
    if status_code is None:
        return "Unknown"
    statuses = [name for bit, name in PRINTER_STATUS_MAP.items() if status_code & bit]
    return ", ".join(statuses) if statuses else "Ready"

def get_jobs(printer_name):
    try:
        handle = win32print.OpenPrinter(printer_name)
        jobs = win32print.EnumJobs(handle, 0, 999, 1)
        win32print.ClosePrinter(handle)
        return jobs
    except Exception:
        return []

def get_printer_status(printer_name):
    try:
        handle = win32print.OpenPrinter(printer_name)
        info = win32print.GetPrinter(handle, 2)
        win32print.ClosePrinter(handle)
        return info['Status']
    except Exception:
        return None

last_job_status = None
last_printer_status = None

while True:
    jobs = get_jobs(target_printer_name)
    printer_status_code = get_printer_status(target_printer_name)
    printer_status_str = decode_printer_status(printer_status_code)

    if jobs:
        job = jobs[0]
        job_id = job['JobId']
        job_status_code = job['Status']
        job_status_str = decode_job_status(job_status_code)
    else:
        job_id = None
        job_status_str = "No jobs"

    # Only print if there is a change
    if (job_status_str != last_job_status) or (printer_status_str != last_printer_status):
        print(f"Printer status: {printer_status_str} | Job status: {job_status_str}")
        last_job_status = job_status_str
        last_printer_status = printer_status_str

    time.sleep(2)

Printer status: Unknown | Job status: No jobs


KeyboardInterrupt: 

In [10]:
# Get status code of all printers
import win32print
printer_list = win32print.EnumPrinters(win32print.PRINTER_ENUM_LOCAL | win32print.PRINTER_ENUM_CONNECTIONS)
printer_status_codes = []

for printer in printer_list:
    printer_name = printer[2]
    try:
        handle = win32print.OpenPrinter(printer_name)
        info = win32print.GetPrinter(handle, 2)
        status_code = info['Status']
        win32print.ClosePrinter(handle)
    except Exception:
        status_code = None
    printer_status_codes.append((printer_name, status_code))

for name, code in printer_status_codes:
    print(f"Printer: {name} | Status code: {code}")

Printer: KONICA MINOLTA 205i | Status code: 0
Printer: OneNote (Desktop) - Protected | Status code: 0
Printer: OneNote (Desktop) | Status code: 0
Printer: NPID5E410 (HP LaserJet Tank MFP 2606sdw) | Status code: 0
Printer: Microsoft Print to PDF | Status code: 0
Printer: HP44F8D2 (HP Smart Tank 660-670 series) | Status code: 1048576
Printer: HP Smart Tank 660-670 series | Status code: 0
Printer: HP LaserJet 1018 | Status code: 0
Printer: ECE_Dept_Printer_1 (HP LaserJet MFP M233dw) | Status code: 0
Printer: Brother DCP-T820DW | Status code: 0


In [14]:
import win32print

# Define printer status constants manually (from Windows API documentation)
PRINTER_STATUS_READY = 0x00000000
PRINTER_STATUS_OFFLINE = 0x00000080
PRINTER_STATUS_PAPER_OUT = 0x00000020
PRINTER_STATUS_PAPER_JAM = 0x00000008
PRINTER_STATUS_ERROR = 0x00000002
PRINTER_STATUS_NOT_AVAILABLE = 0x00001000
PRINTER_ATTRIBUTE_OFFLINE = 0x00000400  # For checking Attributes field

# Dictionary to map status codes to human-readable descriptions
status_map = {
    PRINTER_STATUS_READY: "Ready",
    PRINTER_STATUS_OFFLINE: "Offline",
    PRINTER_STATUS_PAPER_OUT: "Out of Paper",
    PRINTER_STATUS_PAPER_JAM: "Paper Jam",
    PRINTER_STATUS_ERROR: "Error",
    PRINTER_STATUS_NOT_AVAILABLE: "Not Available",
    1048576: "Offline (Driver Detected)"  # Custom mapping for your observed status
}

# Get all printers
printer_list = win32print.EnumPrinters(win32print.PRINTER_ENUM_LOCAL | win32print.PRINTER_ENUM_CONNECTIONS)
printer_status_codes = []

for printer in printer_list:
    printer_name = printer[2]
    try:
        # Open printer handle
        handle = win32print.OpenPrinter(printer_name)
        # Get detailed printer info (level 2)
        info = win32print.GetPrinter(handle, 2)
        status_code = info['Status']
        attributes = info['Attributes']
        port_name = info.get('PortName', 'Unknown')
        
        # Check if the printer is likely virtual (e.g., PDF or OneNote)
        is_virtual = any(keyword in printer_name.lower() for keyword in ['pdf', 'onenote', 'fax'])
        
        # Infer connectivity status
        connectivity = "Unknown"
        if is_virtual:
            connectivity = "Virtual Printer"
        elif port_name == "Unknown" or not port_name:
            connectivity = "No Port (Likely Offline)"
        elif "USB" in port_name or "WSD" in port_name or "LPT" in port_name:
            # Check if the port suggests a physical connection
            if status_code == PRINTER_STATUS_OFFLINE or attributes & PRINTER_ATTRIBUTE_OFFLINE:
                connectivity = "Offline (Physical)"
            else:
                connectivity = "Connected (Physical)"
        elif "IP_" in port_name or "://" in port_name:
            connectivity = "Network Printer (Check IP)"
        
        # Close the printer handle
        win32print.ClosePrinter(handle)
    except Exception as e:
        status_code = None
        port_name = "Unknown"
        connectivity = f"Error: {str(e)}"
    
    printer_status_codes.append((printer_name, status_code, port_name, connectivity))

# Print results
for name, code, port, conn in printer_status_codes:
    status_desc = status_map.get(code, "Unknown Status") if code is not None else "Error"
    print(f"Printer: {name} | Status: {status_desc} (Code: {code}) | Port: {port} | Connectivity: {conn}")

Printer: KONICA MINOLTA 205i | Status: Ready (Code: 0) | Port: Unknown | Connectivity: No Port (Likely Offline)
Printer: OneNote (Desktop) - Protected | Status: Ready (Code: 0) | Port: Unknown | Connectivity: Virtual Printer
Printer: OneNote (Desktop) | Status: Ready (Code: 0) | Port: Unknown | Connectivity: Virtual Printer
Printer: NPID5E410 (HP LaserJet Tank MFP 2606sdw) | Status: Ready (Code: 0) | Port: Unknown | Connectivity: No Port (Likely Offline)
Printer: Microsoft Print to PDF | Status: Ready (Code: 0) | Port: Unknown | Connectivity: Virtual Printer
Printer: HP44F8D2 (HP Smart Tank 660-670 series) | Status: Offline (Driver Detected) (Code: 1048576) | Port: Unknown | Connectivity: No Port (Likely Offline)
Printer: HP Smart Tank 660-670 series | Status: Ready (Code: 0) | Port: Unknown | Connectivity: No Port (Likely Offline)
Printer: HP LaserJet 1018 | Status: Ready (Code: 0) | Port: Unknown | Connectivity: No Port (Likely Offline)
Printer: ECE_Dept_Printer_1 (HP LaserJet MFP M2

In [16]:
import win32print
import win32con

# Define printer status constants (from Windows API)
PRINTER_STATUS_READY = 0x00000000
PRINTER_STATUS_OFFLINE = 0x00000080
PRINTER_STATUS_PAPER_OUT = 0x00000020
PRINTER_STATUS_PAPER_JAM = 0x00000008
PRINTER_STATUS_ERROR = 0x00000002
PRINTER_STATUS_NOT_AVAILABLE = 0x00001000
PRINTER_ATTRIBUTE_OFFLINE = 0x00000400

# Dictionary to map status codes to human-readable descriptions
status_map = {
    PRINTER_STATUS_READY: "Ready",
    PRINTER_STATUS_OFFLINE: "Offline",
    PRINTER_STATUS_PAPER_OUT: "Out of Paper",
    PRINTER_STATUS_PAPER_JAM: "Paper Jam",
    PRINTER_STATUS_ERROR: "Error",
    PRINTER_STATUS_NOT_AVAILABLE: "Not Available",
    1048576: "Offline (Driver Detected)"  # Your observed status
}

# Function to check if printer has active jobs (indicates responsiveness)
def check_printer_jobs(handle):
    try:
        jobs = win32print.EnumJobs(handle, 0, -1, 1)
        return len(jobs) > 0  # True if there are active jobs
    except:
        return False

# Get all printers
printer_list = win32print.EnumPrinters(win32print.PRINTER_ENUM_LOCAL | win32print.PRINTER_ENUM_CONNECTIONS)
printer_status_codes = []

for printer in printer_list:
    printer_name = printer[2]
    try:
        # Open printer handle
        handle = win32print.OpenPrinter(printer_name)
        # Get detailed printer info (level 2)
        info = win32print.GetPrinter(handle, 2)
        status_code = info['Status']
        attributes = info['Attributes']
        port_name = info.get('PortName', 'Unknown')
        
        # Check if the printer is likely virtual
        is_virtual = any(keyword in printer_name.lower() for keyword in ['pdf', 'onenote', 'fax'])
        
        # Infer connectivity status
        connectivity = "Unknown"
        if is_virtual:
            connectivity = "Virtual Printer"
        elif port_name == "Unknown" or not port_name:
            connectivity = "No Port (Likely Offline)"
        elif "USB" in port_name:
            # For USB printers, check status, attributes, and job activity
            if status_code == PRINTER_STATUS_OFFLINE or attributes & PRINTER_ATTRIBUTE_OFFLINE:
                connectivity = "Offline (Physical)"
            elif check_printer_jobs(handle):
                connectivity = "Connected (Active Jobs)"
            else:
                # Additional check: try to get printer state via DeviceIoControl (if supported)
                try:
                    state = win32print.DeviceIoControl(handle, 0x4100000C, None, 1024)
                    connectivity = "Connected (Responsive)" if state else "Offline (Unresponsive)"
                except:
                    connectivity = "Connected (USB Detected)"
        elif "WSD" in port_name or "LPT" in port_name:
            connectivity = "Offline (Physical)" if status_code == PRINTER_STATUS_OFFLINE else "Connected (Physical)"
        elif "IP_" in port_name or "://" in port_name:
            connectivity = "Network Printer (Check IP)"
        
        # Close the printer handle
        win32print.ClosePrinter(handle)
    except Exception as e:
        status_code = None
        port_name = "Unknown"
        connectivity = f"Error: {str(e)}"
    
    printer_status_codes.append((printer_name, status_code, port_name, connectivity))

# Print results
for name, code, port, conn in printer_status_codes:
    status_desc = status_map.get(code, "Unknown Status") if code is not None else "Error"
    print(f"Printer: {name} | Status: {status_desc} (Code: {code}) | Port: {port} | Connectivity: {conn}")

Printer: KONICA MINOLTA 205i | Status: Ready (Code: 0) | Port: Unknown | Connectivity: No Port (Likely Offline)
Printer: OneNote (Desktop) - Protected | Status: Ready (Code: 0) | Port: Unknown | Connectivity: Virtual Printer
Printer: OneNote (Desktop) | Status: Ready (Code: 0) | Port: Unknown | Connectivity: Virtual Printer
Printer: NPID5E410 (HP LaserJet Tank MFP 2606sdw) | Status: Ready (Code: 0) | Port: Unknown | Connectivity: No Port (Likely Offline)
Printer: Microsoft Print to PDF | Status: Ready (Code: 0) | Port: Unknown | Connectivity: Virtual Printer
Printer: HP44F8D2 (HP Smart Tank 660-670 series) | Status: Offline (Driver Detected) (Code: 1048576) | Port: Unknown | Connectivity: No Port (Likely Offline)
Printer: HP Smart Tank 660-670 series | Status: Ready (Code: 0) | Port: Unknown | Connectivity: No Port (Likely Offline)
Printer: HP LaserJet 1018 | Status: Ready (Code: 0) | Port: Unknown | Connectivity: No Port (Likely Offline)
Printer: ECE_Dept_Printer_1 (HP LaserJet MFP M2

In [17]:
!pip install pyusb

    numpy (>=1.19.*) ; python_version >= "3.7"
           ~~~~~~~^

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import usb.core
import usb.util
import sys

def find_printer_port():
    # Find all USB devices
    devices = usb.core.find(find_all=True)
    
    if not devices:
        print("No USB devices found.")
        return
    
    # Iterate through all USB devices
    for device in devices:
        try:
            # Get device information
            manufacturer = usb.util.get_string(device, device.iManufacturer) or "Unknown"
            product = usb.util.get_string(device, device.iProduct) or "Unknown"
            serial = usb.util.get_string(device, device.iSerialNumber) or "Unknown"
            port = device.port_number if hasattr(device, 'port_number') else "Unknown"
            
            # Check if the device is likely a printer (USB class 7 is for printers)
            if device.bDeviceClass == 7 or (device.bDeviceClass == 0 and any(
                cfg.bInterfaceClass == 7 for cfg in device.configurations for interface in cfg)):
                print(f"Printer Found:")
                print(f"  Manufacturer: {manufacturer}")
                print(f"  Product: {product}")
                print(f"  Serial: {serial}")
                print(f"  USB Port: {port}")
                print(f"  Vendor ID: {hex(device.idVendor)}")
                print(f"  Product ID: {hex(device.idProduct)}")
                print("-" * 50)
                
        except Exception as e:
            print(f"Error accessing device: {e}")
            continue

def main():
    print("Scanning for USB printers...")
    try:
        find_printer_port()
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Ensure you have sufficient permissions (e.g., run as root on Linux).")
        print("Also, ensure 'pyusb' is installed and the USB device is connected.")

if __name__ == "__main__":
    main()

Scanning for USB printers...
An error occurred: No backend available
Ensure you have sufficient permissions (e.g., run as root on Linux).
Also, ensure 'pyusb' is installed and the USB device is connected.


In [2]:
pip show pyusb

Name: pyusb
Version: 1.3.1
Summary: Easy USB access for Python
Home-page: https://pyusb.github.io/pyusb
Author: Jonas Malaco
Author-email: me@jonasmalaco.com
License: 
Location: c:\users\prakh\appdata\local\programs\python\python310\lib\site-packages
Requires: 
Required-by: #N/A
Note: you may need to restart the kernel to use updated packages.


In [3]:
import usb.core
import usb.util
import sys

def find_printer_port():
    try:
        # Find all USB devices
        devices = usb.core.find(find_all=True)
        if not devices:
            print("No USB devices found. Ensure your printer is connected and powered on.")
            return

        # Iterate through devices
        for device in devices:
            try:
                # Get device information
                manufacturer = usb.util.get_string(device, device.iManufacturer) or "Unknown"
                product = usb.util.get_string(device, device.iProduct) or "Unknown"
                serial = usb.util.get_string(device, device.iSerialNumber) or "Unknown"
                port = device.port_number if hasattr(device, 'port_number') else "Unknown"

                # Check for printer (USB class 7 or interface class 7)
                is_printer = (device.bDeviceClass == 7 or
                              (device.bDeviceClass == 0 and any(
                                  cfg.bInterfaceClass == 7 for cfg in device.configurations
                                  for interface in cfg)))

                # Debugging: Print all devices to help identify the printer
                print(f"Device: {manufacturer} {product}")
                print(f"  Vendor ID: {hex(device.idVendor)}")
                print(f"  Product ID: {hex(device.idProduct)}")
                print(f"  Serial: {serial}")
                print(f"  Port: {port}")
                print(f"  Is Printer: {'Yes' if is_printer else 'No'}")
                print("-" * 50)

            except Exception as e:
                print(f"Error accessing device {manufacturer} {product}: {e}")
                continue

    except Exception as e:
        print(f"Error initializing USB backend: {e}")
        print("Ensure libusb is installed via Zadig and you have sufficient permissions.")

def main():
    print("Scanning for USB printers...")
    find_printer_port()
    print("If your printer wasn't detected, check Device Manager or provide its Vendor/Product ID.")

if __name__ == "__main__":
    main()

Scanning for USB printers...
Error initializing USB backend: No backend available
Ensure libusb is installed via Zadig and you have sufficient permissions.
If your printer wasn't detected, check Device Manager or provide its Vendor/Product ID.


In [ ]:
import subprocess

def run_powershell_ciminstance():
    # PowerShell script to find printers
    ps_script = """
    Write-Host "Scanning for printers using CIM..."
    Get-CimInstance Win32_PnPEntity | Where-Object { $_.Name -match 'Print(er)?' } | ForEach-Object {
        Write-Host "Printer Found:"
        Write-Host "  Name: $($_.Name)"
        Write-Host "  Port: $($_.PortName -eq $null ? 'Unknown' : $_.PortName)"
        Write-Host "  Device ID: $($_.DeviceID)"
        Write-Host "  Status: $($_.Status)"
        Write-Host "-" * 50
    }
    """

    # Write the PowerShell script to a temporary file
    with open("find_printer.ps1", "w") as f:
        f.write(ps_script)

    try:
        # Run the PowerShell script
        result = subprocess.run(
            ["powershell", "-ExecutionPolicy", "Bypass", "-File", "find_printer.ps1"],
            capture_output=True,
            text=True,
            check=True
        )
        print("PowerShell Output:")
        print(result.stdout)
        if result.stderr:
            print("PowerShell Errors:")
            print(result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"Error running PowerShell script: {e}")
        print(f"Error Output: {e.stderr}")
    except FileNotFoundError:
        print("PowerShell not found. Ensure PowerShell is installed and accessible.")

if __name__ == "__main__":
    print("Running PowerShell script to find printer ports...")
    run_powershell_ciminstance()

Scanning for printers (Windows)...
Printer Found:
  Name: Root Print Queue
Error: <unknown>.PortName
